在window上折腾了一晚上，被小伙伴嘲笑，说谁还在window下搞spark分布式啊。于是又花了差不多一上午的时间在ubuntu下搭了个spark环境，并将其和ipython上交互。以后还是在ubuntu下学习吧，那些命令不用就特别容易忘记。
配置spark开发环境参考的两个主要博客如下，照着这两个博客下去，没有什么坑，亲测有效

[ubuntu spark环境搭建](http://blog.csdn.net/u010171031/article/details/51849562)

[ubuntu anaconda ipython 配置](http://blog.csdn.net/duxu24/article/details/53587451)

[降低pyspark输出日志级别](http://blog.csdn.net/huanbia/article/details/51315889)

我的spark　scala jdk 安装目录在/usr/lib/spark　　/scala  /jdk 下，记一下，免得以后想用忘记在哪了

期间碰到的一些的忘记的linux命令
+ 将一个文件由一个文件夹移入另一个文件夹：在待移文件夹路径下　sudo　mv file 直接写待移到的路径下
+ 给某个文件赋予权限　sudo chmod 777 file
+ 解压zip文件　sudo unzip file

In [11]:
# 开发环境测试
lines = sc.textFile('/home/lee/test.txt')#代表文件中各行文本的RDD 
bLines = lines.filter(lambda line:'b' in line)
print 'this is a transform',bLines
print lines.count()
print bLines.first()
print lines.first()
sc

this is a transform PythonRDD[30] at RDD at PythonRDD.scala:43
5
bbb
aaa


## spark　核心概念简介
＋　**driver program** 驱动器程序 用来发起集群上的各种并行操作。包含应用的main函数，并且定义了集群上的分布式数据集，还对这些分布式数据集应用了相关操作

+ **SparkContext** 驱动器程序通过SparkContext对象来访问Spark.这个对象代表对计算集群的一个连接。shell启动时已经自动创建了一个SparkContext对象，即sc

+ **RDD** 在SparkContext基础上创建起来的。

+ **Spark编程的核心概念**　通过一个驱动程序创建一个SparkContext和一系列RDD,然后进行并行操作　

## 初始化SparkContext
+ Step1: 先创建一个SparkConf对象配置应用
+ Step2:　基于Step1的SparkConf创建一个SparkContext对象


In [8]:
# 交互环境下无需这样　不然会报错
from pyspark import SparkConf,SparkContext
conf = SparkConf().setMaster('local').setAppName('My_App')
sc = SparkContext(conf = conf)

## 构建独立应用
这个只有scala java版，不知道为什么没有python版

## RDD 弹性分布式数据集
RDD可以看成一个简单的“数组”，与一般数组的区别是：RDD是分布式存储，可以更好的利用现有的云数据平台，并在内存中运行。简单的说RDD就是分布式的元素集合（不可变的分布式对象集合）。在Spark中，对数据的所有操作不外乎创建RDD　转化已有RDD以及调用RDD操作进行求值。Spark会自动将RDD中的数据分发到集群上，并将操作并行化执行
### 1. 创建RDD
+ 读取一个外部数据集
+ 在驱动器程序里分发驱动器程序的对象集合（list set）进行并行化，这是RDD创建最简单的一个方式，即将一个已有的集合传给SparkContext的parallelize()方法。这个一般用在开发原型和测试时，因为这种方式需要把整个数据集先放在一台机器的内存中
### 2. RDD两种类型的操作
+ transformation 转化操作　由一个RDD生成一个新的RDD,转化出来的RDD是惰性求值的，只有在行动操作中用到这些RDD时才会被计算。如map() filter(),返回值类型是RDD
+ action 行为操作 会对RDD计算出一个结果，必然会用到RDD转化操作。并将结果返回到驱动器程序中，或把结果存储到外部存储系统中（HDFS ）,会触发实际计算，如count() first(),返回的是其他数据类型

**注意**：虽然可以在任何时候定义新的RDD,但spark只会惰性计算这些RDD.默认情况下，Spark的RDD会在每次对他们进行行动操作时重新计算。若想在多个行动中重用同一个RDD,可以使用RDD.persist()让spark把这个RDD缓存下来（放在内存中）。

### 3. 惰性求值
意味着在被调用行动操作之前Spark不会开始计算。Spark会在内部记录下所要求执行的操作相关信息，从这个角度看，我们可以把RDD看作我们通过转化操作构建出来的　记录如何计算数据的指令列表。把数据读取到RDD的操作同样也是惰性的。

**优势**　可以把一些操作合并到一起来减少计算数据的步骤。

### 4. 向Spark传递函数
Spark的大部分转化操作和一部分行动操作，都需要依赖用户传递函数来计算。python中的三种函数传递方式
+ lambda表达式　传递比较短的函数
+ 传递顶层函数
+ 定义局部函数

### 5. 常见的转化操作和行动操作
#### 基本RDD (支持任意RDD数据类型)
+ 针对各个元素的转化操作
  + map()
  + flatMap() 将函数应用于RDD 中每个元素，将返回的迭代器所有内容展平，构成新的RDD,通常用来切分单词
+ 伪集合操作　支持合并于相交，但要求操作的RDD是相同的数据类型

  + distinct() rdd1.distinct() 生成一个只包含不同元素的新RDD 但操作开销很大，因为会将所有数据通过的网络进行混洗
  + union() rdd1.union(rdd2) 并集操作,与数学中不同的是，即使是相同的元素也会重复包含在并集中
  + intersection()　rdd1.intersection()　交集操作　去除重复元素，效率很差，也要经过混洗
  + substract()　rdd1.subtract(rdd2) 差集操作　也需要数据混洗
  + cartesian() rdd1.cartesian(rdd2) 笛卡尔积　返回所有可能的（a,b）对，在考虑所有可能组合的相似度时比较有用，但在大规模RDD的笛卡尔积开销巨大
+ 行动操作
  + reduce() 并行整合RDD中所有数据　接受一个函数作为参数，这个函数要操作两个RDD的元素类型的数据并返回一个同样类型的新元素。常用来对RDD中的元素进行累加
  + fold() 和reduce一样，但需要提供初始值　接受一个与reduce()接收的函数签名相同的函数，再加上一个’初始值‘来作为每个分区第一次调用时的结果。所提供的初始值应当是提供操作的单位元素，即使用你的函数对这个初始值进行多次计算不会改变结果
  + aggregate() 和reduce类似，但是通常返回不同类型的函数。需要提供我们期待返回的类型的初始值，然后通过一个函数吧RDD中的元素合并起来放入累加器。考虑到每个节点是在本地进行累加，最终，还需要把第二个函数将累加器两两合并
  + countByValue() 各元素在RDD中出现的次数，以字典的形式返回
  + top(n) 测试表明返回排序后的top(n)
  + takeOrdered(num)　从rDD中按照提供的顺序返回最前面的num个元素
  + foreach() 对RDD中所有元素应用一个行动操作，但是不把任何结果返回到驱动器程序中
+ 不同RDD 类型间的转换
mean() variance()只能用在数值RDD上，join()只能用在键值对RDD上

### 持久化


In [10]:
# parallelize()方法
lines = sc.parallelize(['pandas','i love pandas'])
lines.first()

'pandas'

In [18]:
# lambda 函数
inputRDD = sc.textFile('/home/lee/test.txt')
errorRDD = inputRDD.filter(lambda x: 'error' in x)
#print errorRDD.count()
warningRDD = inputRDD.filter(lambda x: 'warning' in x)
#warningRDD.count()
badLinesRDD = errorRDD.union(warningRDD)
badLinesRDD.count()

# 使用take获取RDD中少量元素
for line in badLinesRDD.take(5):
    print line
    
# collect()用来获取整个RDD中数据　不能用在大规模数据集上，只有当数据集能在单台机器的内存中放得下的时候
# 大多数情况下，RDD不能通过collect()收集到驱动程序中，此时，我们通常把数据写到诸如HDFS 这样的分布式存储系统中
# 可以使用saveAsTextFile() saveAsSequenceFile(),或者任意其他行动操作把RDD数据内容以各自自带的格式保存起来


# 自定义函数
def containsError(s):
    return 'error' in s
word = rdd.filter(containsError)

# 局部函数　
# 传递函数时需要小心一点，python会在不经意间把函数所在的对象也序列化传出去。当你传递的对象是某个对象的
# 成员时，或者包含老对某个对象中一个字段的引用时，Spark就会把整个对象发到工作节点上，这可能比你想
# 传递的东西大得多

# 传递一个带字段引用的函数的正确打开方式
class SearchFunctions(objects):
    def __init__(self,query):
        self.query = query
    def isMatch(self,s):
        return self.query in s
    def getMatchesNoReference(self,rdd):
        query = self.query # 非常重要，一定要把所需字段从对象中拿出来放到一个局部变量中，然后传递这个局部变量
        return rdd.filter(lambda x : self.query in x)


aaa,error
cccerror
error,error
bad error
error,warning


In [46]:
# 5中转化操作和行动操作的example
## 5.1 map
nums = sc.parallelize([1,2,3,4])
squared = nums.map(lambda x: x*x).collect()
for num in squared:
    print '%i'%num
    
## 5.1flatMap() 对每个输入元素生成多个输出元素　flatMap函数被分别应用到输入RDD的每个元素上
## 返回一个返回值序列的迭代器，即将各个迭代器均展平，得到一个包含各个迭代器可访问的所有元素的RDD 
## 常用来将输入的字符串切分为单词
lines = sc.parallelize(['hello spark','hi' ,'Hi,spark'])
words = lines.flatMap(lambda line:line.split(' ')).collect()
for word in words:
    print word
#words.first()

# rdd = sc.parallelize([1,2,3,3])
# # ss = rdd.flatMap(lambda x:x.to(3)).collect()#报错了。。有经验再回来研究
# # for x in ss:
# #     print x

# ss = rdd.sample(False,0.5).collect()
# for i in ss:
#     print i #没有输出。。

rdd = sc.parallelize([19,1,7,4,5])
sum1 = rdd.reduce(lambda x,y:x+y)
sum2 = rdd.fold((2),lambda x,y:x+y)# 都不知道如何查文档。。为什么会这样？
sum2
#rdd.top(3)
rdd.takeOrdered(2) # 什么鬼　我要查文档。。

1
4
9
16
hello
spark
hi
Hi,spark


[1, 4]

In [9]:
# from pyspark import  SparkContext
# def isprime(n):
#     """
#     check if integer n is a prime
#     """
#     # make sure n is a positive integer
#     n = abs(int(n))
#     # 0 and 1 are not primes
#     if n < 2:
#         return False
#     # 2 is the only even prime number
#     if n == 2:
#         return True
#     # all other even numbers are not primes
#     if not n & 1:
#         return False
#     # range starts with 3 and only needs to go up the square root of n
#     # for all odd numbers
#     for x in range(3, int(n**0.5)+1, 2):
#         if n % x == 0:
#             return False
#     return True

#     # Create an RDD of numbers from 0 to 1,000,000
#     nums = sc.parallelize(xrange(1000000))

#     # Compute the number of primes in the RDD
#     print nums.filter(isprime).count()
    
# isprime(7)


True